In [151]:
import openai
import os
import pickle
from dotenv import load_dotenv
from pprint import pprint

In [152]:
load_dotenv()

True

In [153]:
new_conversation = True
prompt = """Use Chrome instead"""

In [154]:
model = "gpt-4-1106-preview"
input_cost_usd_per_1K_tokens = 0.01
output_cost_usd_per_1K_tokens = 0.03

API_KEY = os.environ["API_KEY"]
openai.api_key = API_KEY

In [155]:
def prompt_gpt(
    model: str = model,
    messages: list[str] = [""],
) -> str:
    """
    Useful links:
    ---
    models: https://platform.openai.com/docs/models/gpt-3-5
    account balance: https://platform.openai.com/account/billing/overview
    create params: https://platform.openai.com/docs/api-reference/chat/create
    pricing: https://openai.com/pricing
    """

    response = openai.ChatCompletion.create(model=model, messages=messages)
    msg = response.get("choices")[0].get("message").get("content")

    assistant = {"role": "assistant", "content": msg}
    messages.append(assistant)
    
    input_tokens = response.get("usage").get("prompt_tokens")
    output_tokens = response.get("usage").get("prompt_tokens")

    return {
        "msg": msg, 
        "messages": messages, 
        "input_tokens": input_tokens, 
        "output_tokens": output_tokens
        }

In [164]:
def calculate_cost(input_tokens: int, output_tokens: int):
    input_tokens_thousands = input_tokens / 1000
    output_tokens_thousands = output_tokens / 1000
    
    input_cost = input_tokens_thousands * input_cost_usd_per_1K_tokens
    output_cost = output_tokens_thousands * output_cost_usd_per_1K_tokens
    
    print("Tokens used:", input_tokens + output_tokens)
    print("Prompt cost USD:", input_cost + output_cost)

In [157]:
role = "You are an expert Python programmer"
messages = [
    {"role": "system", "content": role},
    {"role": "user", "content": prompt},
]

In [158]:
if not new_conversation:
    if not os.path.exists("messages.pkl"):
        print("No history to load")
    else:
        with open("messages.pkl", mode="rb") as m:
            messages = pickle.load(m)
            messages.append({"role": "user", "content": prompt})

No history to load


In [165]:
result = prompt_gpt(messages=messages)
calculate_cost(input_tokens=result.get("input_tokens"), output_tokens=result.get("output_tokens"))

Tokens used: 734
Prompt cost USD: 0.014679999999999999


In [160]:
pprint(result.get("msg"))

("The error message you're seeing suggests that the Firefox WebDriver process "
 'initiated by Selenium has unexpectedly terminated. Here are some common '
 'reasons why this might happen and steps you can take to troubleshoot the '
 'issue:\n'
 '\n'
 '1. GeckoDriver and Firefox Compatibility:\n'
 '   Ensure that the version of GeckoDriver and Firefox you have installed is '
 'compatible. Sometimes, new versions of Firefox require a corresponding '
 'update of GeckoDriver. Visit the official [GeckoDriver releases '
 'page](https://github.com/mozilla/geckodriver/releases) to download the '
 'latest version.\n'
 '\n'
 '2. Corrupted Firefox Profile:\n'
 '   A corrupted Firefox profile can cause the WebDriver to fail. Try creating '
 'a new Firefox profile or resetting the existing one.\n'
 '\n'
 '3. Missing or Incorrect Path to GeckoDriver:\n'
 '   Confirm that GeckoDriver is installed and that its executable is in your '
 "system's PATH, or specify its path directly in your script. You c

In [161]:
with open("messages.pkl", mode="wb") as f:
    pickle.dump(result.get("msgs"), file=f)